In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import sys
sys.path.append('/home/aliche/ownCloud/Workspaces/Eclipse/dataAnalysis/Sleep-current/src/')

In [3]:
%matplotlib
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as pl
import pickle as pkl
import fnmatch,os
import itertools
import statsmodels.formula.api as sm
from scipy.stats import pearsonr,ttest_ind,ranksums,linregress,ks_2samp,entropy,mannwhitneyu
import pandas as pd
import seaborn as sns
from signale.tools import locate
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches

Using matplotlib backend: Qt5Agg
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
sparseness = lambda x: np.power(np.mean(np.abs(x)),2)/ np.mean(np.power(x,2))#lambda a:(np.sqrt(a.size)- np.linalg.norm(a,1)/ np.linalg.norm(a,2)) / (np.sqrt(a.size)-1)
l_max      = lambda x: np.math.pow(1 + np.math.sqrt(1.0*x.shape[0]/x.shape[1]),2)

In [5]:
sns.set_style('darkgrid')
def jointplot_w_hue(data, x, y, hue=None, colormap = None, 
                    figsize = None, fig = None, scatter_kws=None):
    #defaults
    if colormap is None:
        colormap = sns.color_palette() #['blue','orange']
    if figsize is None:
        figsize = (5,5)
    if fig is None:
        fig  = plt.figure(figsize = figsize)
    if scatter_kws is None:
        scatter_kws = dict(alpha=0.4, lw=1)
        
    # derived variables
    if hue is None:
        return "use normal sns.jointplot"
    hue_groups = data[hue].unique()

    subdata = dict()
    colors = dict()
    
    active_colormap = colormap[0: len(hue_groups)]
    legend_mapping = []
    for hue_grp, color in zip(hue_groups, active_colormap):
        legend_entry = mpatches.Patch(color=color, label=hue_grp)
        legend_mapping.append(legend_entry)
        
        subdata[hue_grp] = data[data[hue]==hue_grp]
        colors[hue_grp] = color
        
    # canvas setup
    grid = gridspec.GridSpec(2, 2,
                           width_ratios=[4, 1],
                           height_ratios=[1, 4],
                           hspace = 0, wspace = 0
                           )
    ax_main    = plt.subplot(grid[1,0])
    ax_xhist   = plt.subplot(grid[0,0], sharex=ax_main)
    ax_yhist   = plt.subplot(grid[1,1])#, sharey=ax_main)
    
    ## plotting
   
    # histplot x-axis
    for hue_grp in hue_groups:
        sns.distplot(subdata[hue_grp][x], color = colors[hue_grp]
                     , ax = ax_xhist)

    # histplot y-axis
    for hue_grp in hue_groups:
        sns.distplot(subdata[hue_grp][y], color = colors[hue_grp]
                     , ax = ax_yhist, vertical=True) 

    # main scatterplot 
    # note: must be after the histplots else ax_yhist messes up
    for hue_grp in hue_groups:
        sns.regplot(data = subdata[hue_grp], fit_reg=False,
                    x = x, y = y, ax = ax_main, color = colors[hue_grp]
                    , scatter_kws=scatter_kws
                   )        
        
    # despine
    for myax in [ax_yhist, ax_xhist]:
        sns.despine(ax = myax, bottom=False, top=True, left = False, right = True
                    , trim = False)
        plt.setp(myax.get_xticklabels(), visible=False)
        plt.setp(myax.get_yticklabels(), visible=False)
    
    
    # topright 
    ax_legend   = plt.subplot(grid[0,1])#, sharey=ax_main)
    plt.setp(ax_legend.get_xticklabels(), visible=False)
    plt.setp(ax_legend.get_yticklabels(), visible=False)

    ax_legend.legend(handles=legend_mapping)
    #plt.close()
    return dict(fig = fig, gridspec = grid)

In [6]:
###100ms
les100rn = sorted([os.path.join(item[0],item[1])for item in locate('*100ms-vis.lsn','../PCA_MEC/gammas/')])
cnt100rn = sorted([os.path.join(item[0],item[1])for item in locate('*100ms-vis.cnt','../PCA_MEC/gammas/')])
###Theta
lesThrn = sorted([os.path.join(item[0],item[1])for item in locate('*-theta-vis.lsn','../PCA_MEC/gammas/')])
cntThrn = sorted([os.path.join(item[0],item[1])for item in locate('*-theta-vis.cnt','../PCA_MEC/gammas/')])
spikeDATApath = '/home/aliche/dataWork/ali/MECLesion_SleepData/'
dataFolder    = '/home/aliche/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/' 
spkFiles = sorted([os.path.join(item[0],item[1])for item in locate('FiringUnits.spk',spikeDATApath)])
trajDBs = sorted([os.path.join(item[0],item[1])for item in locate('linearTrack*.db',dataFolder) if item[0].find('trajectory')>0])
trajFLs = sorted([os.path.join(item[0],item[1])for item in locate('*.traj',spikeDATApath)])

In [7]:
spDB = pd.DataFrame([],columns=['phi','sp','group'])
for pth in les100rn[:]:
    Thpth = pth.replace('100ms','theta')
    info = pth.split('/')[-1].split('-')
    anID = info[0]
    Dday   = info[1]
    DD = Dday[0]+Dday[-1]+'.db'
    spkPth = [spkpth  for spkpth in spkFiles if (spkpth.find(Dday)>0 and spkpth.find(anID)>0) ][0]
    trjPth = [trj  for trj in trajFLs if (trj.find(Dday)>0 and trj.find(anID)>0) ][0]
    dbPth = [db  for db in trajDBs if (db.find(DD)>0 and db.find(anID)>0) ][0]
    Nslices = 1 #number of slices that a theta cycle should be cut
    parrentFolder = spkPth[:-15]
    stList = pd.read_pickle(spkPth)
    stList.changeTimeUnit('ms')
    traj   = pd.read_pickle(trjPth)
    #Interpolating trajectory
    t = traj.times
    It = np.arange(0,traj.times.size)
    Itp = np.arange(0,traj.times.size,0.15)
    tInterp = np.interp(Itp,It,t) #interpolated traj times
    
    sig100 = stList.id_list.size - pd.read_pickle(pth).shape[1]
    sigtet = stList.id_list.size - pd.read_pickle(pth.replace('100ms','theta')).shape[1]
    if Nslices==1:
        tt = pkl.load(open(parrentFolder+'thBins.npArr','rb'))
        bb = 100
    else:
        tt = pkl.load(open(parrentFolder+'4thBins.npArr','rb'))
        bb = 25
    pre,track,post  = [stList.time_slice(item[0],item[-1]) for item in tt[:3]]
    immo = stList.time_slice(tt[1][0],tt[1][-1])
    ExperimentSessions = [pre,track,immo,post]
    for sessID,sess in enumerate(ExperimentSessions):
        for _id,cell in enumerate(sess):
            if cell.spike_times.size ==0:
                print sessID,_id
                cell.spike_times = np.append(cell.spike_times,sess.first_spike_time())
    
    #track.changeTimeUnit('s')
    #immo.changeTimeUnit('s')
    tdb = pd.read_pickle(dbPth)
    ####################################################################################
    for _idx_,cell in enumerate(track):
        #print '_________%s_________' %_idx_
        immSPKids = np.zeros_like(cell.spike_times)
        for _ii_ in range(tdb.shape[0]):
            immSPKids+= np.logical_and(cell.spike_times<tInterp[tdb[_ii_,0]]*1e3,cell.spike_times>tInterp[tdb[_ii_-1,1]]*1e3)
        immSPKids = immSPKids.astype(bool)
        immo[_idx_].spike_times = cell.spike_times[immSPKids]
        runSPKids = np.logical_not(immSPKids)
        track[_idx_].spike_times = cell.spike_times[runSPKids]
    immoTT = np.zeros_like(tt[1])
    for _ii_ in range(tdb.shape[0]):
        immoTT+= np.logical_and(tt[1] < tInterp[tdb[_ii_,0]]*1e3,tt[1] > tInterp[tdb[_ii_-1,1]]*1e3)
    immoTT = immoTT.astype(bool)
    #TT = [tt[0],tt[1][np.logical_not(immoTT)],tt[1][immoTT],tt[2]]
    TT = [tt[0],tt[1],tt[1],tt[2]]
    ###################################################################################
    print spkPth
    corrMatrices = []
    Qs = []
    eigVals = []
    eigVecs = []
    Ps = []
    #track.changeTimeUnit('ms')
    #immo.changeTimeUnit('ms')
    for sessID,sess in enumerate(ExperimentSessions):
        for _id,cell in enumerate(sess):
            if cell.spike_times.size ==0:
                print sessID,_id
                cell.spike_times = np.append(cell.spike_times,sess.t_start+np.random.randint(10,1000))
    for ii,item in enumerate(ExperimentSessions):
        print ii
        Q,C,l,V,P = item.PCA(bb)
        corrMatrices.append(C)
        Qs.append(Q)
        eigVals.append(l)
        eigVecs.append(V)
        Ps.append(P)
    #####PCA with time bins defined by theta periods
    ThetaCorrMatrices = []
    ThetaQs = []
    ThetaEigVals = []
    ThetaEigVecs = []
    ThetaPs = []
    for ii,item in enumerate(ExperimentSessions):
        print ii
        Q,C,l,V,P = item.PCA(bins=TT[ii])
        ThetaCorrMatrices.append(C)
        ThetaQs.append(Q)
        ThetaEigVals.append(l)
        ThetaEigVecs.append(V)
        ThetaPs.append(P)
    xdots = np.abs(ThetaEigVals[1]/l_max(ThetaQs[1]))
    ydots = np.abs([sparseness(vec) for vec in ThetaEigVecs[1]])
    df = pd.DataFrame(np.transpose([xdots[sigtet:],ydots[sigtet:],xdots[sigtet:].size*['Les.']]),columns=spDB.columns)
    spDB = spDB.append(df,ignore_index=True)
    #pl.plot(xdots[sigtet:],ydots[sigtet:],'r.')

IndexError: list index out of range

In [8]:
for pth in cnt100rn:
    Thpth = pth.replace('100ms','theta')
    info = pth.split('/')[-1].split('-')
    anID = info[0]
    Dday   = info[1]
    DD = Dday[0]+Dday[-1]+'.db'
    spkPth = [spkpth  for spkpth in spkFiles if (spkpth.find(Dday)>0 and spkpth.find(anID)>0) ][0]
    trjPth = [trj  for trj in trajFLs if (trj.find(Dday)>0 and trj.find(anID)>0) ][0]
    dbPth = [db  for db in trajDBs if (db.find(DD)>0 and db.find(anID)>0) ][0]
    Nslices = 1 #number of slices that a theta cycle should be cut
    parrentFolder = spkPth[:-15]
    stList = pd.read_pickle(spkPth)
    stList.changeTimeUnit('ms')
    traj   = pd.read_pickle(trjPth)
    #Interpolating trajectory
    t = traj.times
    It = np.arange(0,traj.times.size)
    Itp = np.arange(0,traj.times.size,0.15)
    tInterp = np.interp(Itp,It,t) #interpolated traj times
    
    sig100 = stList.id_list.size - pd.read_pickle(pth).shape[1]
    sigtet = stList.id_list.size - pd.read_pickle(pth.replace('100ms','theta')).shape[1]
    if Nslices==1:
        tt = pkl.load(open(parrentFolder+'thBins.npArr','rb'))
        bb = 100
    else:
        tt = pkl.load(open(parrentFolder+'4thBins.npArr','rb'))
        bb = 25
    pre,track,post  = [stList.time_slice(item[0],item[-1]) for item in tt[:3]]
    immo = stList.time_slice(tt[1][0],tt[1][-1])
    ExperimentSessions = [pre,track,immo,post]
    for sessID,sess in enumerate(ExperimentSessions):
        for _id,cell in enumerate(sess):
            if cell.spike_times.size ==0:
                print sessID,_id
                cell.spike_times = np.append(cell.spike_times,sess.first_spike_time())
    
    #track.changeTimeUnit('s')
    #immo.changeTimeUnit('s')
    tdb = pd.read_pickle(dbPth)
    ####################################################################################
    for _idx_,cell in enumerate(track):
        #print '_________%s_________' %_idx_
        immSPKids = np.zeros_like(cell.spike_times)
        for _ii_ in range(tdb.shape[0]):
            immSPKids+= np.logical_and(cell.spike_times<tInterp[tdb[_ii_,0]]*1e3,cell.spike_times>tInterp[tdb[_ii_-1,1]]*1e3)
        immSPKids = immSPKids.astype(bool)
        immo[_idx_].spike_times = cell.spike_times[immSPKids]
        runSPKids = np.logical_not(immSPKids)
        track[_idx_].spike_times = cell.spike_times[runSPKids]
    immoTT = np.zeros_like(tt[1])
    for _ii_ in range(tdb.shape[0]):
        immoTT+= np.logical_and(tt[1] < tInterp[tdb[_ii_,0]]*1e3,tt[1] > tInterp[tdb[_ii_-1,1]]*1e3)
    immoTT = immoTT.astype(bool)
    #TT = [tt[0],tt[1][np.logical_not(immoTT)],tt[1][immoTT],tt[2]]
    TT = [tt[0],tt[1],tt[1],tt[2]]
    ###################################################################################
    print spkPth
    corrMatrices = []
    Qs = []
    eigVals = []
    eigVecs = []
    Ps = []
    #track.changeTimeUnit('ms')
    #immo.changeTimeUnit('ms')
    for sessID,sess in enumerate(ExperimentSessions):
        for _id,cell in enumerate(sess):
            if cell.spike_times.size ==0:
                print sessID,_id
                cell.spike_times = np.append(cell.spike_times,sess.t_start+np.random.randint(10,1000))
    for ii,item in enumerate(ExperimentSessions):
        print ii
        Q,C,l,V,P = item.PCA(bb)
        corrMatrices.append(C)
        Qs.append(Q)
        eigVals.append(l)
        eigVecs.append(V)
        Ps.append(P)
    #####PCA with time bins defined by theta periods
    ThetaCorrMatrices = []
    ThetaQs = []
    ThetaEigVals = []
    ThetaEigVecs = []
    ThetaPs = []
    for ii,item in enumerate(ExperimentSessions):
        print ii
        Q,C,l,V,P = item.PCA(bins=TT[ii])
        ThetaCorrMatrices.append(C)
        ThetaQs.append(Q)
        ThetaEigVals.append(l)
        ThetaEigVecs.append(V)
        ThetaPs.append(P)
    xdots = np.abs(ThetaEigVals[1]/l_max(ThetaQs[1]))
    ydots = np.abs([sparseness(vec) for vec in ThetaEigVecs[1]])
    df = pd.DataFrame(np.transpose([xdots[sigtet:],ydots[sigtet:],xdots[sigtet:].size*['Cnt.']]),columns=spDB.columns)
    spDB = spDB.append(df,ignore_index=True)

IndexError: list index out of range

In [31]:
vecs = ThetaEigVecs[-1][sigtet:]
f,axarr = pl.subplots(1,vecs.shape[0])
for ii in range(vecs.shape[0]):
    item = vecs[ii,:].reshape(vecs.shape[1],1)
    axarr[ii].pcolor(item)
    axarr[ii].set_xlabel(np.round(sparseness(item),2),fontsize=24)
    pl.setp(axarr[ii].get_xticklabels(), visible=False)
    pl.setp(axarr[ii].get_yticklabels(), visible=False)
    pl.setp(axarr[ii].get_xticklines(),visible=False)
    pl.setp(axarr[ii].get_yticklines(),visible=False)

In [192]:
spDB.phi = spDB.phi.astype('float')
spDB.sp = spDB.sp.astype('float')
f,ax = pl.subplots(1,1)
sns.set(style="white", color_codes=True)
jointplot_w_hue(x='phi',y='sp',data=spDB,hue='group',fig=f,colormap=sns.xkcd_palette(['red','black']))
pl.show()

In [182]:
spDB = spDB.replace('Lesioned','Les.')
spDB = spDB.replace('Control','Cnt.')

In [186]:
spDB

,phi,sp,group
0,1.319435,0.6160770150286851,Les.
1,1.572182,0.3365895696944864,Les.
2,1.727824,0.378971126040835,Les.
3,2.470746,0.5862098853659478,Les.
4,1.212115,0.6678265414178153,Les.
5,1.287884,0.5321835670143048,Les.
6,1.347761,0.4834849724783154,Les.
7,1.376579,0.5951589174013576,Les.
8,1.599437,0.6366095563822061,Les.
9,1.121790,0.6292078305600313,Les.
